In [106]:
import pickle

# Recuperamos la lista de "hiperparámetros" con diferencias
# estadísticamnete significativas
with open("test7", "rb") as f:
    list_df_signif = pickle.load(f)

En esta lista tenemos tuplas con las matrices que hay por cada preproceso, trozo y canal. Estas tuplas tienen la forma:
* DataFrame con la matriz correspondiente
* int que indica el tipo de preproceso
* int que indica el trozo de la grabación
* int que indica el canal

### Clasificación SVM

In [107]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import svm
import math
import pandas as pd

In [108]:
grid = {'kernel': ['linear','poly','rbf','sigmoid'],
        'C': [0.5,1,2,3],
        'gamma': ['scale','auto'],
        'random_state': [12]
        }

In [109]:
list_results_svm = []
cv_results_svm = []
for data in list_df_signif:
    (df_data, prep, troz, chan) = data
    print('prep:',prep,'troz:', troz,'chan:', chan)
    data_arr = df_data.iloc[:,:-1].to_numpy()
    label_arr = df_data['Label'].to_numpy()
    svc = svm.SVC()
    clf = GridSearchCV(estimator=svc, param_grid=grid, cv=5, return_train_score=True,
                               verbose=2)
    clf.fit(data_arr, label_arr)
    list_results_svm.append(clf)
    cv_results_svm.append((clf.cv_results_, prep, troz, chan))

prep: 0 troz: 0 chan: 0
Fitting 5 folds for each of 32 candidates, totalling 160 fits
[CV] END .C=0.5, gamma=scale, kernel=linear, random_state=12; total time=   0.0s
[CV] END .C=0.5, gamma=scale, kernel=linear, random_state=12; total time=   0.0s
[CV] END .C=0.5, gamma=scale, kernel=linear, random_state=12; total time=   0.0s
[CV] END .C=0.5, gamma=scale, kernel=linear, random_state=12; total time=   0.0s
[CV] END .C=0.5, gamma=scale, kernel=linear, random_state=12; total time=   0.0s
[CV] END ...C=0.5, gamma=scale, kernel=poly, random_state=12; total time=   0.0s
[CV] END ...C=0.5, gamma=scale, kernel=poly, random_state=12; total time=   0.0s
[CV] END ...C=0.5, gamma=scale, kernel=poly, random_state=12; total time=   0.0s
[CV] END ...C=0.5, gamma=scale, kernel=poly, random_state=12; total time=   0.0s
[CV] END ...C=0.5, gamma=scale, kernel=poly, random_state=12; total time=   0.0s
[CV] END ....C=0.5, gamma=scale, kernel=rbf, random_state=12; total time=   0.0s
[CV] END ....C=0.5, gam

In [110]:
def conversor_num_to_chan(num):
    if num == 0:
        a = 'Fp1'
    elif num == 1:
        a = 'Fp2'
    elif num == 2:
        a = 'F3'
    elif num == 3:
        a = 'F4'
    elif num == 4:
        a = 'C3'
    elif num == 5:
        a = 'C4'
    elif num == 6:
        a = 'P3'
    elif num == 7:
        a = 'P4'
    elif num == 8:
        a = 'O1'
    elif num == 9:
        a = 'O2'
    elif num == 10:
        a = 'F7'
    elif num == 11:
        a = 'F8'
    elif num == 12:
        a = 'T7'
    elif num == 13:
        a = 'T8'
    elif num == 14:
        a = 'P7'
    elif num == 15:
        a = 'P8'
    elif num == 16:
        a = 'Fz'
    elif num == 17:
        a = 'Cz'
    elif num == 18:
        a = 'Pz'
    else:
        print('error de canales')
    return a

In [111]:
list_accuracies_svm = []
for (dic_res,prep,troz,chan) in cv_results_svm:
    df = pd.DataFrame(dic_res)
    ind_max = df['mean_test_score'].idxmax()
    acc = df.iloc[ind_max]['mean_test_score']
    std_error = df.iloc[ind_max]['std_test_score']
    param_C = df.iloc[ind_max]['param_C']
    param_gamma = df.iloc[ind_max]['param_gamma']
    param_kernel = df.iloc[ind_max]['param_kernel']
    lab_chan = conversor_num_to_chan(chan)
    list_accuracies_svm.append((prep, troz, lab_chan, param_C, param_gamma, \
                                param_kernel, acc, std_error))

In [112]:
list_accuracies_svm.sort(key=lambda x: x[6], reverse=True)
df_results_svm = pd.DataFrame(list_accuracies_svm)

In [113]:
df_results_svm.columns = ['preproceso', 'trozo', 'canal', \
                          'C', 'gamma', 'kernel', 'accuracy',\
                          'std_error']
df_results_svm

,preproceso,trozo,canal,C,gamma,kernel,accuracy,std_error
0,0,0,F8,1.0,scale,rbf,0.768000,0.069052
1,0,0,Fz,2.0,scale,rbf,0.752000,0.095099
2,0,0,P3,2.0,auto,rbf,0.744667,0.092960
3,0,0,T7,0.5,scale,rbf,0.743667,0.096721
4,0,0,Pz,1.0,scale,rbf,0.727667,0.099279
5,0,0,Fp2,0.5,scale,rbf,0.727000,0.122947
6,0,0,P7,0.5,scale,rbf,0.711333,0.099763
7,0,0,Fp1,0.5,scale,sigmoid,0.709667,0.110209
8,0,0,O1,2.0,scale,poly,0.704667,0.148412
9,0,1,Fp1,3.0,scale,linear,0.701667,0.114431


In [94]:
#list_results_svm[0].best_params_

{'C': 0.5, 'gamma': 'scale', 'kernel': 'sigmoid', 'random_state': 12}

In [20]:
# list_best_params_svm = []
# for best_p in list_results_svm:
#     list_best_params_svm.append(best_p.best_params_)

In [44]:
#list_accuracies_svm.sort(reverse=True)

In [45]:
#list_accuracies_svm

[(0.7270000000000001, 2, 0, 0, 1),
 (0.7096666666666667, 3, 0, 0, 0),
 (0.6856666666666668, 15, 0, 0, 2)]

In [50]:
#list_results_svm[1].best_params_

{'C': 0.5, 'gamma': 'scale', 'kernel': 'rbf', 'random_state': 12}

In [ ]:
# list_good_res = []
# for tup in list_accuracies_svm:
#     acc = tup[0]
#     pos = tup[1]
#     prep = tup[2]
#     troz = tup[3]
#     chan = tup[4]
#     print(pos)
#     dic = list_results_svm[pos].best_params_
#     dic['preproceso'] = prep
#     dic['trozo'] = troz
#     dic['channel'] = chan
#     dic['accuracy'] = acc
#     list_good_res.append(dic)
# df_results_svm = pd.DataFrame(list_good_res)

In [20]:
# a = list(df_results_svm['channel'])
# b = [conversor_num_to_chan(i) for i in a]
# df_results_svm['channel'] = b

In [23]:
df_results_svm.head(5)
# 1 trozo, 1 sensor, solo caract DES, svm

,C,gamma,kernel,random_state,preproceso,trozo,channel,accuracy
0,0.5,scale,rbf,12,0,31,P7,0.810333
1,2.0,scale,rbf,12,0,14,Cz,0.809667
2,2.0,scale,poly,12,0,29,Fz,0.809667
3,2.0,scale,rbf,12,0,20,F8,0.809333
4,1.0,scale,sigmoid,12,0,17,F8,0.809333


In [9]:
df_results_svm.head(50)
# antiguo

,C,gamma,kernel,random_state,preproceso,trozo,channel,accuracy
0,1.0,scale,linear,12,0,26,16,0.843684
1,3.0,scale,rbf,12,0,0,12,0.832632
2,2.0,scale,sigmoid,12,0,17,16,0.832632
3,3.0,scale,rbf,12,0,29,16,0.823158
4,1.0,auto,sigmoid,12,0,13,16,0.823158
5,3.0,scale,rbf,12,0,23,14,0.822632
6,1.0,scale,rbf,12,0,12,18,0.813158
7,0.5,auto,sigmoid,12,0,23,18,0.812105
8,1.0,auto,sigmoid,12,0,19,18,0.811579
9,3.0,auto,sigmoid,12,0,28,12,0.811579


In [22]:
with open("results_DES_svm", "wb") as fp:   #Pickling
    pickle.dump(df_results_svm, fp)

Estos resultados están guardados en `results_DES_svm`